In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from numpy import nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

from sklearn.utils import resample
plt.style.use('classic')
%matplotlib inline
pd.set_option('display.max_rows', None)

In [ ]:
df_train1 = pd.read_csv('sample_data/X_trainData_1.csv')
df_train2 = pd.read_csv('sample_data/Y_trainData_1.csv')
df_test = pd.read_csv('sample_data/X_testData_1.csv')
df_test = df_test.loc[:, df_test.columns != 'ID']

In [ ]:
#temp = df_train1.isin([0]).sum(axis=0)
#temp

In [ ]:
#output = df_train2.drop_duplicates()
#output.groupby('Unnamed: 0').size()
print(df_train1.shape)
print(df_train2.shape)
print(df_test.shape)

(3393, 222)
(3393, 11)
(1000, 222)


In [ ]:
res = pd.concat([df_train1, df_train2], axis=1)
res = res[res.OFLX!= -1]
res2=res[res.OFLX==1]
# res3=res[res.OFLX==0]
# df_upsampled = resample(res3,replace=True, n_samples=203,random_state=123)   
# res=pd.concat([res2, df_upsampled], axis=0)
y = res['OFLX']
X = res.drop(['OFLX'], axis = 1) 
X = X.drop(['INH'], axis = 1) 
X = X.drop(['RIF'], axis = 1) 
X = X.drop(['CIP'], axis = 1) 
X = X.drop(['PZA'], axis = 1) 
X = X.drop(['EMB'], axis = 1)
X = X.drop(['CAP'], axis = 1)
X = X.drop(['STR'], axis = 1)
X = X.drop(['AMK'], axis = 1)
X = X.drop(['MOXI'], axis = 1)  
X = X.drop(['KAN'], axis = 1)  

print(y.shape)
print(X.shape)
print(res.shape)
print(res2.shape)
# print(y)
# print(X)
# upsamp = SMOTE(ratio='minority')
# X_sm, y_sm = upsamp.fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1)


(690,)
(690, 222)
(690, 233)
(603, 233)


In [1]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# SVC Code
# defining parameter range 
param_grid = {'C': [0.01,0.1,1,2,10,15,5],  
              'gamma': [0.1,0.01,1,10,0.2,0.05], 
              'kernel': ['rbf']}  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2,scoring='roc_auc') 
# # fitting the model for grid search 
grid.fit(X_train, y_train) 
print(grid.best_params_) #Use these to train the SVClassifier
model = SVC(C=9, kernel='rbf', degree=3, gamma=0.01,probability=True)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_OFLX.csv')
ID = df['ID'].tolist()
# print(y_pred.shape)
# print(type(y_pred))
# print(len(y_pred))

NameError: ignored

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num =50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000, num = 101)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
print(rf_random.best_params_) #Use these to train the RF model
model = RandomForestClassifier(n_estimators=567, criterion='entropy', max_depth=208, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split=2, bootstrap=False, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_OFLX.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

(1000, 2)


In [ ]:

cv_params = {'max_depth': [3,4,5,6], 'min_child_weight': [1,2], 'subsample': [0.8,0.9,1], 'max_delta_step': [0,1,2], 'learning_rate':  [0.2, 0.25, 0.3,0.1]}  
gscv = GridSearchCV(xgb.XGBClassifier(), cv_params, scoring = 'roc_auc', cv = 5)#running grid search cv on model 
gscv.fit(X_train, y_train)#fitting model
# gscv.grid_scores_#displaying scores
best_params= gscv.best_params_
print(best_params)#use these to train the XGB Classifier
model = xgb.XGBClassifier(learning_rate=0.30,max_depth=10,min_child_weight=1, eval_metric='auc',max_delta_step=3,scale_pos_weight=1.1)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)#for converting into probabilities
df = pd.read_csv('sample_data/Y_testData_1_nolabels_OFLX.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

(1000, 2)


In [ ]:
li_id = []
li_val = []
for i in ID:
    # print(i, y_pred[i-1])
    li_id.append(i)
    li_val.append((y_pred[i-1][1]))
csv = pd.DataFrame({'ID': li_id, 'OFLX' :li_val})
csv.to_csv('OFLX-submit.csv', index=False, header=True)  
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))  
print(roc_auc_score(y_test, y_pred))

[[ 6  6]
 [ 0 57]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.90      1.00      0.95        57

    accuracy                           0.91        69
   macro avg       0.95      0.75      0.81        69
weighted avg       0.92      0.91      0.90        69

0.75
